# LangChain Agents Tutorial

This notebook demonstrates how to create and use agents in LangChain. We'll build a ReAct agent that can search the web and get weather information.

## Step 1: Setting up the OpenAI API Key

First, we need to set up our OpenAI API key to use the language model.


In [1]:
from google.colab import userdata
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

## Step 2: Installing Required Packages

We need to install the necessary LangChain packages and other dependencies for our agent.


In [2]:
!pip install -q langchain-groq==0.3.* langchain-community==0.3.* langchain-core==0.3.* requests duckduckgo-search ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [3]:
!pip show langchain-groq langchain-community langchain-core requests duckduckgo-search ddgs

Name: langchain-groq
Version: 0.3.8
Summary: An integration package connecting Groq and LangChain
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: groq, langchain-core
Required-by: 
---
Name: langchain-community
Version: 0.3.31
Summary: Community contributed LangChain integrations.
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: aiohttp, dataclasses-json, httpx-sse, langchain, langchain-core, langsmith, numpy, pydantic-settings, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
---
Name: langchain-core
Version: 0.3.79
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: jsonpatch, langsmith, packaging, pydantic, PyYAML, tenacity, typing-extensions
Required-by: langchain, langchain-community, langchain-groq, langchain-text-splitters
---
Name: reque

## Step 3: Importing Core Libraries

Let's import the essential LangChain components and other libraries we'll need.


In [4]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
import requests

## Step 4: Setting up the Search Tool

We'll use DuckDuckGo search as one of our tools to allow the agent to search for information on the web.


In [5]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

## Step 5: Creating a Custom Weather Tool

Here we define a custom tool that can fetch weather data for any city using the WeatherStack API. This tool will be available to our agent.


In [ ]:
@tool
def get_weather_data(city: str) -> str:
  """
  This tool fetches the current weather data for a given city
  """
  url = f'https://api.weatherstack.com/current?access_key=[WEATHERSTACK_API_KEY]&query={city}' # Enter the weatherstack api key before use

  response = requests.get(url)

  return response.json()

## Step 6: Initializing the Language Model

We create an instance of the ChatOpenAI model that will power our agent's reasoning capabilities.


In [8]:
# llm = ChatGroq(model="openai/gpt-oss-20b", temperature=0)
llm = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0,
    # Enable tool calling for this model
    model_kwargs={"tool_choice": "auto"},
    api_key=GROQ_API_KEY
)

## Step 7: Importing Agent Components

We import the necessary components to create a ReAct agent and agent executor.


In [9]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

## Step 8: Loading the ReAct Prompt

We pull the standard ReAct (Reasoning and Acting) prompt from LangChain Hub. This prompt template guides the agent on how to reason through problems and take actions.


In [10]:
# Step 2: Pull the ReAct prompt from LangChain Hub
base_prompt = hub.pull("hwchase17/react")  # pulls the standard ReAct agent prompt

In [11]:
# Add stopping condition to prevent infinite loop

stopping_instructions = """
You will reason step-by-step using the format:
Thought: <your reasoning>
Action: <tool name>
Action Input: <input to the tool>

**Stopping condition:**
If you have gathered all required information to answer the user's question,
STOP calling more tools.
Instead, print your answer starting with 'Final Answer:' on a new line.
Do NOT print any more Thoughts or Actions after the final answer.

Example:
Thought: I should find the capital of Maharashtra.
Action: search_tool
Action Input: "capital of Maharashtra"

Thought: I found the info I need.
Final Answer: The capital of Maharashtra is Mumbai.
"""

In [12]:
modified_prompt = base_prompt + "\n" + stopping_instructions # add stopping condition to the predefined prompt

## Step 9: Creating the ReAct Agent

Now we create our ReAct agent by combining the language model, our tools (search and weather), and the ReAct prompt template.


In [13]:
# Step 3: Create the ReAct agent manually with the pulled prompt
agent = create_react_agent(
    llm=llm,
    tools=[search_tool, get_weather_data],
    prompt=modified_prompt
)

## Step 10: Creating the Agent Executor

The AgentExecutor wraps our agent and handles the execution loop, managing tool calls and responses.


In [14]:
# Step 4: Wrap it with AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool, get_weather_data],
    verbose=True,
    handle_parsing_errors=True
)

## Step 11: Testing the Agent

Let's test our agent with a complex query that requires both searching for information and getting weather data. The agent will need to:
1. Find the capital of Madhya Pradesh
2. Get the current weather for that city


In [15]:
# Step 5: Invoke
response = agent_executor.invoke({"input": "Find the capital of Madhya Pradesh, then find it's current weather condition"})
# response = agent_executor.invoke({"input": "Find the weather condition of Bhopal."})
print(response)



> Entering new AgentExecutor chain...
Question: Find the capital of Madhya Pradesh, then find it's current weather condition
Thought: I need to find the capital of Madhya Pradesh. I will search for that information.
Action: duckduckgo_search
Action Input: "capital of Madhya Pradesh"Bombay State was enlarged by the addition of Saurashtra State and Kutch State, the Marathi-speaking districts of Nagpur division of Madhya Pradesh and the Marathwada region of Hyderabad State. Bho pal (Hindi: Bhōpāl, pronounced [bʱoːpaːlᵊ] ⓘ) is the capital city of the Indian state of Madhya Pradesh and the administrative headquarters of both Bhopal district and Bhopal division. [10] While most know it as the capital city of Madhya Pradesh , Bhopal district is much more — a complete destination that invites tourists of every kind. Its capital is Bhopal, and Madhya Pradesh is the second-largest Indian state by area and the fifth-largest state by population with over 72 million residents. While most know it 

In [16]:
response['output']

'The capital of Madhya Pradesh is **Bhopal**.  \nCurrent weather in Bhopal (as reported in the latest online source) is **partly cloudy** with a temperature of about **23\u202f°C** (feels like **26\u202f°C**), light east‑ward wind of roughly **0.8\u202fm/s**, humidity around **84\u202f%**, and atmospheric pressure near **716\u202fmm\u202fHg**.'

## Summary

This notebook demonstrated how to:
- Set up LangChain agents with custom tools
- Use the ReAct framework for reasoning and acting
- Combine multiple tools (search and weather API) in a single agent
- Execute complex multi-step queries that require both information retrieval and data processing

The agent successfully found that Bhopal is the capital of Madhya Pradesh and retrieved its current weather conditions by using both the search tool and the weather API tool in sequence.
